# STA 220 Data & Web Technologies for Data Analysis

### Lecture 11, 2/11/25, Natural language processing


### Announcements 

- HW 3 is published. 

### Last week's topics

- Tokenizing
- Tagging
- Topic Modelling

### Today's topics
- Natural Language Processing
    - Naive Bayes
    - ROC space

The goal of today is to answer the following questions:
1. How can we identify particular features of language data that are salient for classifying it?
2. How can we construct models of language that can be used to perform language processing tasks automatically?
3. What can we learn about language from these models?

### Naive Bayes Classifier

We aim to find the probability for a label. The following algorithm first uses the Bayes rule to express $P(label|features)$ in terms of $P(label)$ and $P(features|label)$, where the latter are approximated from the test data set: 
$$
P(label|features) = \frac{P(features|label)P(label)}{P(features)}
$$

As usual for discrete data, the probability in the denominator is not computed directly. Instead, the numerator is normalized. 

Now, we make the 'naive' assumption that all features are independent, given the label:
$$
P(label|features) \propto P(label) P(feature\ 1|label) \dots P(feature\ n|label)
$$

To exemplify, we will consider a simple case first, a single feature. 

In [1]:
from nltk.corpus import names
import nltk
import random
import numpy as np

In [2]:
names = ([(name, 'male') for name in names.words('male.txt')] + 
         [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

In [3]:
names[:4]

[('Anica', 'female'),
 ('Milly', 'female'),
 ('Mandie', 'female'),
 ('Damara', 'female')]

In [4]:
def gender_features(word):
    return word[-1]

In [5]:
gender_features("Trinity")

'y'

In [6]:
len(names)

7944

In [7]:
featuresets = [({'feature': gender_features(n)}, g) for n, g in names]
train_set, test_set = featuresets[500:], featuresets[:500]

In [8]:
test_set[:3]

[({'feature': 'a'}, 'female'),
 ({'feature': 'y'}, 'female'),
 ({'feature': 'e'}, 'female')]

First, lets estimate $P(label)$. 

In [9]:
P_label = np.mean([True if g[1] == 'male' else False for g in train_set])
print(P_label)
print(np.mean([True if g[1] == 'female' else False for g in train_set]))

0.3702310585706609
0.629768941429339


In [10]:
def likelihood(feature, label): 
    return np.mean([True if f['feature'] == feature else False 
                    for f, y in train_set if y == label])

In [11]:
likelihood(' ', 'female')

0.00021331058020477816

To compute the posterior, lets establish how many features there are. 

In [12]:
features = {f['feature'] for f, g in train_set}
len(features)

26

In [13]:
#features

In [14]:
posterior_male = {f: P_label * likelihood(f, 'male') for f in features}
posterior_female = {f: (1 - P_label) * likelihood(f, 'female') for f in features}

In [15]:
normalization = {k: posterior_male.get(k, 0) + posterior_female.get(k, 0) for k in set(posterior_female) & set(posterior_male)}
#normalization

In [16]:
posterior_male = {f: P_label * likelihood(f, 'male') / normalization.get(f, 0) for f in features}
posterior_female = {f: (1 - P_label) * likelihood(f, 'female') / normalization.get(f, 0) for f in features}

In [17]:
print(posterior_male.get('r')) # prob of label given feature
print(posterior_female.get('r'))

0.7963800904977375
0.2036199095022625


Our classifier is implemented. How well does it do? 

In [18]:
posterior_female.get(gender_features('Esther')) 

0.2036199095022625

In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [20]:
classifier.prob_classify({'feature': gender_features('Esther')}).prob('male')

0.7947414790021119

In [21]:
classifier.prob_classify({'feature': gender_features('Esther')}).samples()

dict_keys(['male', 'female'])

In [22]:
gender_features('Esther')

'r'

Lets classify [Moby Dick](https://www.gutenberg.org/files/2701/2701-h/2701-h.htm).

In [23]:
import re
import nltk
import numpy as np
import pandas as pd

In [24]:
moby = nltk.corpus.gutenberg.raw("melville-moby_dick.txt")
pattern = r"\s*(?:EXTRACTS|ETYMOLOGY\.|Epilogue|CHAPTER \d+)\s+.+\n*.+[\.!\?\)]\s*"
corpus = re.split(pattern, moby)
corpus.pop(0)
corpus = [re.sub(r"\s+", " ", document).lower() for document in corpus]

In [25]:
len(corpus)

138

From [here](https://digitalcommons.cwu.edu/cgi/viewcontent.cgi?article=1430&context=etd#:~:text=In%20Chapters%201%2D8%2C%2010,role%20of%20%22I%22%20narrator.). 

In [26]:
cetological = [0, 1, 26, 27, 33, 46, 55, 57, 58, 59, 60, 61, 63, 64, \
               68, 75, 76, 77, 78, 80, 81, 85, 86, 87, 89, 90, 91, \
               93, 96, 97, 98, 102, 103, 104, 105, 106, 107]

In [27]:
label = [i not in cetological for i in range(138)]
label = [i for i in map(lambda x: 'story' if x else 'ceto', label)]

In [28]:
label[:10]

['ceto',
 'ceto',
 'story',
 'story',
 'story',
 'story',
 'story',
 'story',
 'story',
 'story']

In [123]:
def tokenizer(document): 
    words = re.findall('\w+', document)
    words = [w[0] for a, w in nltk.pos_tag(words) if w[0] in ['N', 'V', 'J', 'R', 'P']]
    return words

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(tokenizer = tokenizer) 
freq = vec.fit_transform(corpus)

In [125]:
freq

<138x5 sparse matrix of type '<class 'numpy.int64'>'
	with 690 stored elements in Compressed Sparse Row format>

In [126]:
def get_row_as_dict(i, freq): 
    row = freq[i,:].todense().tolist()[0]
    names = vec.get_feature_names_out()
    return {n: r for r, n in zip(row, names)}

In [127]:
get_row_as_dict(134, freq)

{'J': 365, 'N': 841, 'P': 248, 'R': 326, 'V': 634}

In [128]:
featuresets = [(get_row_as_dict(i, freq), label[i]) for i, _ in enumerate(corpus)]

In [129]:
featuresets[0][1]

'ceto'

In [130]:
#random.shuffle(featuresets)
train_set, test_set = featuresets[:90], featuresets[90:]

In [131]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [132]:
print(nltk.classify.accuracy(classifier, test_set))

0.4791666666666667


In [133]:
sum([True for i in label if i=="story"]) / len(label) #benchmark accuracy

0.7318840579710145

In [134]:
fp = fn = tp = tn = 0
for test in test_set: 
    classification = classifier.classify(test[0])
    truth = test[1]
    if classification == truth and truth == 'story': tp += 1
    if classification == truth and truth != 'story': tn += 1
    if classification != truth and truth == 'story': fp += 1
    if classification != truth and truth != 'story': fn += 1

In [135]:
confusion = pd.DataFrame([[tp, fn], [fp, tn]], 
                         columns = ['true story', 'true ceto'], index = ['class story', 'class ceto'])
confusion

,true story,true ceto
class story,13,2
class ceto,23,10


We assign a chapter to be either cetological or storyline depending on the respective probability exceeding $0.5$. But this is an arbitrary decision rule.

Instead, one should consider choosing cetological chapters based on different thresholds. 

| Estimated \ True      | Success        | Failure       |
| ----------- | ----------- | ----------- |
| Success        | $n_{1|1}$, true positive   | $n_{1|2}$, false positive |
| Failure       | $n_{2|1}$, false negative  | $n_{2|2}$, true negative  |

From the values of the confusion matrix the following measures can be calculated for the evaluation of the classification model: 
 - true positive rate (TPR / sensitivity): $\frac{n_{1|1}}{n_{1|1} + n_{2|1}}$
 - false negative rate: $\frac{n_{2|1}}{n_{1|1} + n_{2|1}}$
 - false positive rate (FPR): $\frac{n_{1|2}}{n_{1|2} + n_{2|2}}$
 - true negative rate (specificity): $\frac{n_{2|2}}{n_{1|2} + n_{2|2}}$
 - positive prediction rate (precision): $\frac{n_{1|1}}{n_{1|1} + n_{1|2}}$
 - false detection rate: $\frac{n_{1|2}}{n_{1|1} + n_{1|2}}$
 - negative prediction rate (separability): $\frac{n_{2|2}}{n_{2|1} + n_{2|2}}$
 - misclassification rate: $\frac{n_{2|1}}{n_{2|1} + n_{2|2}}$

In [136]:
confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[1, 0]) #TPR

0.3611111111111111

In [137]:
confusion.iloc[0, 1] / (confusion.iloc[0, 1] + confusion.iloc[1, 1]) #FPR

0.16666666666666666

### ROC Plots

A ROC (receiver operating characteristics) plot plots the FPR on the x-axis onto the TPR on the y-axis. 

The point $(0,0)$ means that a model never classifies a success: $n_{1|1} = n_{1|2} = 0$. 
This way we avoid false positive errors, at the cost of never having any true positive classifications either. 
Point $(0,1)$ corresponds to a perfect classification $n_{2|1} = n_{1|2} = 0$. 

A point is *better in the ROC space* than another point, if its TPR is larger and its FPR smaller than the TPR and FPR of the other point. 

A point below the bisector $x = y$ corresponds to a point which is worse than a uniformly random classification. The bisector corresponds to a uniform random classification. 

We can use classification methods that return decision for the class (e.g., success or failure), or methods that return a probability that the observation belongs to a certain class (e.g., $\pi$). For the latter, we have to set a threshold $\tau\in[0,1]$, to retrieve a true classification. 
Changing the threshold, will change the TPR and FPR. This leads to a *ROC curve*. 

Let $\hat\pi_i\in[0,1]$ be the estimate for probability of the i-th observation is a success. 
Define $f_j$ to be the probability mass function of a random variable $p_j$, $j = 0,1$, where the realizations of $p_j$ are the values $\hat\pi_i$ that correspond to $y_i = j$. Now we can interpret the FPR and TRP as functions of the threshold $\tau$. 

$$
TPR(\tau) = \int_\tau^\infty f_1(x) dx\qquad FPR(\tau) = \int_\tau^{\infty} f_0(x) dx 
$$

This gives the ROC curve $\tau\rightarrow(FPR(\tau), TPR(\tau))$ as a function of $\tau$.

In [138]:
def TPR(tau, classifier, featuresets, target = 'story'): 
    pi = np.array([classifier.prob_classify(chapter[0]).prob(target) for chapter in featuresets if chapter[1] == 'story'])
    norm = len([True for chapter in featuresets if chapter[1] == 'story'])
    tpr = [sum(pi >= t) / norm for t in tau]
    return tpr

In [139]:
TPR([0, 1], classifier, test_set)

[1.0, 0.0]

In [140]:
def FPR(tau, classifier, featuresets, target = 'story'): 
    pi = np.array([classifier.prob_classify(chapter[0]).prob(target) for chapter in featuresets if chapter[1] != 'story'])
    norm = len([True for chapter in featuresets if chapter[1] != 'story'])
    fpr = [sum(pi >= t) / norm for t in tau]
    return fpr

In [141]:
FPR([0, 1], classifier, test_set)

[1.0, 0.0]

In [142]:
tmp = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9, 1] # range of threshold 
df = pd.DataFrame({
    'tau': tmp,
    'tpr': TPR(tmp, classifier, test_set),
    'fpr': FPR(tmp, classifier, test_set)
})

In [143]:
df
df['method']='POS frequency'

In [144]:
import plotly.express as px

fig = px.line(df, x="fpr", y="tpr", text = 'tau', line_shape="hv", color = "method")
fig.update_layout(yaxis_range=[-0.1,1.1], 
                  xaxis_range=[-0.1,1.1])
fig.update_traces(textposition="bottom right")
fig.show()

To compare different classification methods, we can use the area under the curve (AUC). 
One can show that the AUC $= P(p_0 > p_1)$. 
The larger the AUC, the better the method in classifying. 

In [145]:
df = df.iloc[::-1]
np.trapz(df['tpr'], df['fpr']) # AUC

0.6620370370370371

To what can we compare this to? Try: 

- New tokenizer
- New frequency measure
- Combination of both

In [149]:
words = [tokenizer(document) for document in corpus]
words = [w for l in words for w in l] 

dictionary = dict(nltk.FreqDist(words))
dictionary

{'the': 14333,
 'pale': 19,
 'usher': 1,
 'threadbare': 1,
 'in': 4161,
 'coat': 28,
 'heart': 91,
 'body': 110,
 'and': 6413,
 'brain': 37,
 'i': 2126,
 'see': 272,
 'him': 1065,
 'now': 785,
 'he': 1895,
 'was': 1644,
 'ever': 206,
 'dusting': 2,
 'his': 2528,
 'old': 450,
 'lexicons': 1,
 'grammars': 2,
 'with': 1722,
 'a': 4726,
 'queer': 44,
 'handkerchief': 5,
 'mockingly': 1,
 'embellished': 3,
 'all': 1525,
 'gay': 21,
 'flags': 1,
 'of': 6598,
 'known': 80,
 'nations': 12,
 'world': 176,
 'loved': 3,
 'to': 4622,
 'dust': 10,
 'it': 2522,
 'somehow': 44,
 'mildly': 10,
 'reminded': 4,
 'mortality': 1,
 'while': 246,
 'you': 894,
 'take': 137,
 'hand': 213,
 'school': 9,
 'others': 39,
 'teach': 5,
 'them': 474,
 'by': 1201,
 'what': 618,
 'name': 69,
 'whale': 1215,
 'fish': 167,
 'is': 1725,
 'be': 1045,
 'called': 116,
 'our': 207,
 'tongue': 11,
 'leaving': 38,
 'out': 538,
 'through': 233,
 'ignorance': 11,
 'letter': 15,
 'h': 3,
 'which': 648,
 'almost': 195,
 'alone': 4

In [150]:
def tokenizer(document): 
    words = re.findall('\w+', document)
    #words = [w[0] for a, w in nltk.pos_tag(words) if w[0] in ['N', 'V', 'J', 'R', 'P']]
    words = [word for word in words if dictionary[word] > 20]
    return words

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(tokenizer = tokenizer) 
freq = vec.fit_transform(corpus)

In [152]:
featuresets = [(get_row_as_dict(i, freq), label[i]) for i, _ in enumerate(corpus)]

In [153]:
get_row_as_dict(1, freq)

{'a': 0.23820061043249613,
 'about': 0.007215017562551597,
 'above': 0.00550718244628176,
 'according': 0.0,
 'account': 0.013564143175803142,
 'across': 0.0,
 'act': 0.0,
 'added': 0.0,
 'advance': 0.0,
 'aft': 0.0,
 'after': 0.006489768449602373,
 'afterwards': 0.0,
 'again': 0.0035524185888546693,
 'against': 0.0040975213754050185,
 'age': 0.0,
 'ago': 0.0063363560121460025,
 'ah': 0.0,
 'ahab': 0.0,
 'ahead': 0.0,
 'ain': 0.0,
 'air': 0.01815736563986941,
 'alive': 0.0,
 'all': 0.023641935636885818,
 'almost': 0.0,
 'aloft': 0.006028821648092521,
 'alone': 0.0,
 'along': 0.0,
 'alongside': 0.0,
 'already': 0.0,
 'also': 0.014246658821136626,
 'altogether': 0.013762917597392704,
 'always': 0.0,
 'am': 0.005074809017429759,
 'american': 0.006984903735013339,
 'among': 0.016123328101358858,
 'an': 0.016577042069605256,
 'anchor': 0.0,
 'ancient': 0.007323278833258687,
 'and': 0.2552345509651646,
 'another': 0.004345808535849973,
 'answer': 0.0,
 'answered': 0.015153790452101811,
 'any

In [154]:
train_set, test_set = featuresets[:90], featuresets[90:]

In [155]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))

0.375


In [156]:
TPR([0, 1], classifier, test_set)

[1.0, 0.1111111111111111]

In [157]:
FPR([0, 1], classifier, test_set)

[1.0, 0.08333333333333333]

In [158]:
import plotly.graph_objects as go


df1 = pd.DataFrame({
    'tau': tmp,
    'tpr': TPR(tmp, classifier, test_set),
    'fpr': FPR(tmp, classifier, test_set)
})

df1

In [161]:
fig.add_trace(go.Scatter(x=df1["fpr"], y=df1["tpr"], line_shape="hv", name=
                         'common words with tfidf'))
fig.show()